
class PtientBasedMBFVariability():
    def __init__(self, args):
        self.args = args

    def ReadPatientsData(self):
        """for each patient two files must exist: Patient_n_FFR_s_MBF_Territories.vtu
        and Patient_n_FFR_s_MBF_Territory_Labels.dat, first containing the MBF values
        and territory assignments, and the latter including the name of all territories.
        """
        pass

    def ExtractIschemicRegions(self, MBF):
        pass

    def CreatePatientsDataFrame(self):
        pass

    def ViolinPlot(self):
        pass

In [1]:
import os 
import vtk
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utilities import ReadVTUFile, ThresholdInBetween
from vtk.util.numpy_support import vtk_to_numpy